# RAG

In [2]:
import os
import re
import time

from dotenv import load_dotenv
from typing import Dict, List
from jinja2 import Template

from langchain_core.documents.base import Document
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

from langchain_openai.chat_models import ChatOpenAI 


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
load_dotenv(dotenv_path='../src/.env')

True

## Functions

In [4]:
def load_urls(urls_path: str) -> List[str]:
    with open(urls_path, "r", encoding="utf-8") as file:
        urls = [line.strip() for line in file if line.strip()]
    return urls


def load_documents(urls_list: List[str]) -> List[Document]:
    docs = [WebBaseLoader(url).load() for url in urls_list]
    docs_list = [item for sublist in docs for item in sublist]
    print(f"len of documents :{len(docs_list)}")
    return docs_list


def preprocess_documents(documents: List[Document]) -> List[Document]:
    for doc in documents:
        doc.page_content = re.sub(r'\n+', '\n', doc.page_content.strip())
    return documents


def split_documents(documents: List[Document], chunk_size: int, chunk_overlap: int) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    doc_splits = text_splitter.split_documents(documents)
    print(f"length of document chunks generated :{len(doc_splits)}")
    return doc_splits


def read_template_file(template_path: str) -> str:
    with open(template_path, "r") as file:
        template_content = file.read()

    template = Template(template_content).render()
    return template


def create_prompt_template(template: str, input_variables: List[str]) -> PromptTemplate:
    prompt = PromptTemplate(
        template=template,
        input_variables=input_variables,
    )
    return prompt


def format_documents(documents: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in documents)


### Implement the Retriever

In [5]:
urls = load_urls('../data/urls.txt')

In [6]:
documents = load_documents(urls)

len of documents :8


In [7]:
preprocessed_documents = preprocess_documents(documents)

In [8]:
splitted_documents = split_documents(preprocessed_documents, chunk_size=1024, chunk_overlap=200)

length of document chunks generated :156


In [9]:
embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 56073.58it/s]


In [10]:
vectorstore = FAISS.from_documents(
    documents=splitted_documents,
    embedding=embedding_model
)

In [11]:
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

In [12]:
retrieved_documents = retriever.invoke("que necesito para entrar en Tailandia?")

### Implement the Router

In [13]:
router_template = read_template_file('../prompts/router_prompt.jinja')

In [14]:
router_prompt = create_prompt_template(router_template, input_variables=['question'])

In [15]:
llm = ChatOpenAI(model='gpt-4o-mini')

In [16]:
question_router = router_prompt | llm | JsonOutputParser()

In [17]:
start = time.time()

question = "Que necesito para visitar Vietnam?"
print(question_router.invoke({"question": question}))

end = time.time()

print(f"The time required to generate response by Router Chain in seconds:{end - start}")


{'datasource': 'vectorstore'}
The time required to generate response by Router Chain in seconds:0.6525523662567139


### Implement Generate Chain

In [18]:
generator_template = read_template_file('../prompts/generator_prompt.jinja')

In [19]:
generator_prompt = create_prompt_template(generator_template, input_variables=['question', 'context'])

In [20]:
rag_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | generator_prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("Qué necesito para entrar en tailandia?"):
    print(chunk, end="", flush=True)


Para entrar en Tailandia, necesitas los siguientes documentos:

1. **Pasaporte**: Debe estar en vigor y tener una validez de al menos 6 meses desde la fecha de entrada al país.
2. **Billete de avión**: Debes contar con un billete de avión de entrada a Tailandia y otro de salida del país.
3. **Visado**: Si eres ciudadano español y vas a estar menos de 60 días como turista, no necesitas un visado previo. Sin embargo, si planeas quedarte más tiempo o si viajas por otros motivos, deberás tramitar un visado en la embajada o consulado.

Además, si viajas desde una zona donde la fiebre amarilla es endémica, deberás presentar un certificado de vacunación internacional. También es recomendable tener un seguro de viaje para cubrir posibles gastos imprevistos.

In [21]:
for chunk in rag_chain.stream("Qué necesito para obtener un visado en Vietnam?"):
    print(chunk, end="", flush=True)

Para obtener un visado en Vietnam, específicamente la eVisa, necesitas lo siguiente:

1. **Documentación**:
   - Una foto de carné en formato digital.
   - Una foto de la página principal de tu pasaporte en formato digital.
   - Fecha de llegada a Vietnam.
   - Número de tarjeta de crédito o débito para el pago.

2. **Información adicional**:
   - Datos del puerto de entrada y salida.
   - Dirección de tu primer alojamiento en Vietnam.

3. **Validación del pasaporte**:
   - Tu pasaporte debe tener una validez mínima de 6 meses a partir de la fecha de entrada.

4. **Pago**:
   - La eVisa cuesta 25$ para una entrada y 50$ para múltiples entradas.

Es recomendable tramitar la eVisa a través del portal oficial y no a través de agencias intermediarias. Una vez aprobada, recibirás el visado por correo electrónico y es aconsejable imprimirlo para mostrarlo al llegar al país.

### Implement Retrieval Grader

In [22]:
retrieval_grader_template = read_template_file('../prompts/retrieval_grader_prompt.jinja')

In [23]:
retrieval_grader_prompt = create_prompt_template(retrieval_grader_template, input_variables=['question', 'context'])

In [24]:
retrieval_grader = retrieval_grader_prompt | llm | JsonOutputParser()

In [25]:
retrieval_grader = retrieval_grader_prompt | llm | StrOutputParser()

In [29]:
start = time.time()

question = 'Qué necesito para entrar en Tailandia?'
context = retriever.invoke(question)

print(format_documents(context))
print(f'{"-"*20}')

print(retrieval_grader.invoke({"question": question, "context": context}))

end = time.time()
print(f"The time required to generate response by the retrieval grader in seconds:{end - start}")

Billete de avión de entrada y salida de Tailandia
Aunque suene a una obviedad para muchos viajeros, entre los documentos que se necesitan para viajar a Tailandia encontramos también el billete de avión. Para poder realizar este viaje deberás contar tanto con un billete de ida a Tailandia como con un billete de salida del país.
Dado que muchos viajeros van a Tailandia con un plan de viaje bastante abierto y en ocasiones, si se sienten a gusto, optan por ampliar su estancia con un visado, puede que en el momento de comprar el vuelo de ida al país no sepas cuándo te vas a ir.
Una de las soluciones por las que optan muchos viajeros es comprar un vuelo a Singapur, dado que es un destino cercano y el trayecto puede salir barato (puedes conseguir tu billete de avión por 40€), para una fecha cercana a la expiración del permiso de estancia en Tailandia. De esa manera, si finalmente optases por quedarte más tiempo, no perderías mucho dinero y, si te quisieras ir, desde Singapur podrías volar a p